In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Prepare for Evaluating

## Import libraries

In [2]:
!pip install bert-score
!pip install rouge_score
!pip install pycocoevalcap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-10.3

In [3]:
import os
import torch
import json
import numpy as np
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from bert_score import score as bert_score
from transformers import AutoModel, AutoTokenizer
from pycocoevalcap.cider.cider import Cider

## Evaluation

In [4]:
def load_generated_captions(captions_file):
    captions_dict = {}
    with open(captions_file, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) != 2:
                print(f"Warning: Invalid line format in {captions_file}: {line.strip()}")
                continue
            img_name, caption = parts
            captions_dict[img_name] = caption.strip()
    return captions_dict

def load_gt_captions(captions_file):
    gt_captions = {}
    with open(captions_file, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) != 2:
                print(f"Warning: Invalid line format in {captions_file}: {line.strip()}")
                continue
            img_name, caption = parts
            if img_name not in gt_captions:
                gt_captions[img_name] = []
            gt_captions[img_name].append(caption.strip())
    return gt_captions

# Function to get n-grams from a string
def get_ngrams(text, n):
    tokens = text.split()
    return [' '.join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

# Function to debug common n-grams
def debug_ngrams(gts, hyp, max_n=4):
    for n in range(1, max_n+1):
        gts_ngrams = [get_ngrams(gt, n) for gt in gts]
        hyp_ngrams = get_ngrams(hyp, n)
        # Find common n-grams between the hypothesis and each ground truth
        # Note: We're looking for n-grams that appear in at least one GT and in the hypothesis
        common_ngrams = set()
        for gt_ngram_set in gts_ngrams:
            common_ngrams.update(set(gt_ngram_set).intersection(set(hyp_ngrams)))
        print(f"{n}-gram common: {common_ngrams}")

# Evaluation function
def evaluate_captions(generated_captions, gt_captions, dataset_name="Unknown"):
    bleu_scores = {f"BLEU-{i}": [] for i in range(1, 5)}
    rouge_l_scores = []
    bert_p_scores = []
    bert_r_scores = []
    bert_f1_scores = []
    all_gts = {}  # For CIDEr
    all_hyp = {}  # For CIDEr
    smoothing = SmoothingFunction().method1
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=False)
    cider_scorer = Cider()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Initialize PhoBERT for BERTScore (Vietnamese language model)
    phobert = AutoModel.from_pretrained("vinai/phobert-base")
    tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
    from bert_score.utils import model2layers
    if "vinai/phobert-base" not in model2layers:
        model2layers["vinai/phobert-base"] = 12

    for idx, img_name in enumerate(tqdm(gt_captions.keys(), desc=f"Evaluating {dataset_name}")):
        if img_name not in generated_captions:
            print(f"Warning: No generated caption for {img_name}")
            continue
            
        ground_truths = gt_captions[img_name]  # List of reference captions
        generated_caption = generated_captions.get(img_name, "")
        generated_tokens = generated_caption.split()

        # Skip verbose printing for every image to improve performance
        # print(f"Image: {img_name}")
        # print(f"Generated Caption: {generated_caption}")
        # print(f"Ground Truth Captions: {ground_truths}")
        
        if not generated_caption.strip() or not any(gt.strip() for gt in ground_truths):
            print(f"Warning: Empty caption or ground truth detected for {img_name}!")
            continue
            
        # debug_ngrams(ground_truths, generated_caption)  # Comment out for performance

        # Store for CIDEr - properly format for multiple references
        all_gts[idx] = [gt for gt in ground_truths if gt.strip()]
        all_hyp[idx] = [generated_caption if generated_caption.strip() else ""]

        # BLEU-1 to BLEU-4 with multiple references
        reference_tokenized = [gt.split() for gt in ground_truths]
        for i in range(1, 5):
            weights = tuple(1.0/i if j < i else 0 for j in range(4))
            # Calculate BLEU against all references at once (not individually)
            bleu = sentence_bleu(
                reference_tokenized,  # Pass all references at once
                generated_tokens,
                weights=weights,
                smoothing_function=smoothing
            )
            bleu_scores[f"BLEU-{i}"].append(bleu)

        # ROUGE-L with best match among references
        best_rouge_l = 0.0
        for gt in ground_truths:
            rouge_scores = scorer.score(gt, generated_caption)
            best_rouge_l = max(best_rouge_l, rouge_scores['rougeL'].fmeasure)
        rouge_l_scores.append(best_rouge_l)

        # BERTScore with all references
        # bert-score package handles multiple references internally and returns the best scores
        P, R, F1 = bert_score(
            [generated_caption],  # Single hypothesis
            [ground_truths],      # List of references for one image
            model_type="vinai/phobert-base",
            lang="vi",
            device=device,
            use_fast_tokenizer=True,
            rescale_with_baseline=False
        )
        bert_p_scores.append(P.item())
        bert_r_scores.append(R.item())
        bert_f1_scores.append(F1.item())

    # Compute CIDEr
    try:
        cider_score, cider_scores_per_image = cider_scorer.compute_score(all_gts, all_hyp)
        cider_scores = list(cider_scores_per_image)
    except Exception as e:
        print(f"Error computing CIDEr: {e}")
        cider_scores = [0.0] * len(all_gts)

    # Compute average scores
    avg_bleu_scores = {f"BLEU-{i}": np.mean(bleu_scores[f"BLEU-{i}"]) for i in range(1, 5)}
    avg_rouge_l = np.mean(rouge_l_scores)
    avg_bert_p = np.mean(bert_p_scores)
    avg_bert_r = np.mean(bert_r_scores)
    avg_bert_f1 = np.mean(bert_f1_scores)
    avg_cider = np.mean(cider_scores)

    # Print results
    print(f"\nEvaluation Results for {dataset_name}:")
    for i in range(1, 5):
        print(f"Average BLEU-{i}: {avg_bleu_scores[f'BLEU-{i}']:.4f}")
    print(f"Average ROUGE-L: {avg_rouge_l:.4f}")
    print(f"Average BERTScore Precision: {avg_bert_p:.4f}")
    print(f"Average BERTScore Recall: {avg_bert_r:.4f}")
    print(f"Average BERTScore F1: {avg_bert_f1:.4f}")
    print(f"Average CIDEr: {avg_cider:.4f}")

    results = {
        **avg_bleu_scores,
        "ROUGE-L": avg_rouge_l,
        "BERTScore_P": avg_bert_p,
        "BERTScore_R": avg_bert_r,
        "BERTScore_F1": avg_bert_f1,
        "CIDEr": avg_cider
    }
    return results

# Main

## CNN + LSTM

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Running on device: {device}")

# define datasets
datasets = [
    {
        "name": "capydata-ic",
        "gt_captions_file": "/kaggle/input/capydata-ic/data/test/captions.txt",
        "generated_captions_file": "/kaggle/input/test-generated-captions/generated_captions/cnn_lstm/us_generated_captions.txt"
    },
    {
        "name": "test_uit_viic",
        "gt_captions_file": "/kaggle/input/uit-viic-test/test/captions.txt",
        "generated_captions_file": "/kaggle/input/test-generated-captions/generated_captions/cnn_lstm/uit_generated_captions.txt"
    }
]

results = {}
for dataset in datasets:
    # Load captions
    generated_captions = load_generated_captions(dataset["generated_captions_file"])
    gt_captions = load_gt_captions(dataset["gt_captions_file"])
    
    # Evaluate
    results[dataset["name"]] = evaluate_captions(
        generated_captions, gt_captions, dataset["name"]
    )
    
# Compare results
print("\nComparison of Results:")
print(f"{'Metric':<20} {'capydata-ic':<15} {'test_uit_viic':<15}")
for metric in results["capydata-ic"]:
    print(f"{metric:<20} {results['capydata-ic'][metric]:<15.4f} {results['test_uit_viic'][metric]:<15.4f}")

Running on device: cuda


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

2025-05-18 16:03:38.660689: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747584218.867599      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747584218.924947      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]


Evaluating capydata-ic: 100%|██████████| 279/279 [04:50<00:00,  1.04s/it]



Evaluation Results for capydata-ic:
Average BLEU-1: 0.5840
Average BLEU-2: 0.3606
Average BLEU-3: 0.2135
Average BLEU-4: 0.1340
Average ROUGE-L: 0.4760
Average BERTScore Precision: 0.5894
Average BERTScore Recall: 0.5728
Average BERTScore F1: 0.5794
Average CIDEr: 0.3654


Evaluating test_uit_viic: 100%|██████████| 578/578 [10:14<00:00,  1.06s/it]



Evaluation Results for test_uit_viic:
Average BLEU-1: 0.3742
Average BLEU-2: 0.1537
Average BLEU-3: 0.0677
Average BLEU-4: 0.0425
Average ROUGE-L: 0.3986
Average BERTScore Precision: 0.4987
Average BERTScore Recall: 0.5432
Average BERTScore F1: 0.5174
Average CIDEr: 0.0879

Comparison of Results:
Metric               capydata-ic     test_uit_viic  
BLEU-1               0.5840          0.3742         
BLEU-2               0.3606          0.1537         
BLEU-3               0.2135          0.0677         
BLEU-4               0.1340          0.0425         
ROUGE-L              0.4760          0.3986         
BERTScore_P          0.5894          0.4987         
BERTScore_R          0.5728          0.5432         
BERTScore_F1         0.5794          0.5174         
CIDEr                0.3654          0.0879         


## CNN + LSTM (Attention)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Running on device: {device}")

# define datasets
datasets = [
    {
        "name": "capydata-ic",
        "gt_captions_file": "/kaggle/input/capydata-ic/data/test/captions.txt",
        "generated_captions_file": "/kaggle/input/test-generated-captions/generated_captions/cnn_lstm_attention/us_generated_captions.txt"
    },
    {
        "name": "test_uit_viic",
        "gt_captions_file": "/kaggle/input/uit-viic-test/test/captions.txt",
        "generated_captions_file": "/kaggle/input/test-generated-captions/generated_captions/cnn_lstm_attention/uit_generated_captions.txt"
    }
]

results = {}
for dataset in datasets:
    # Load captions
    generated_captions = load_generated_captions(dataset["generated_captions_file"])
    gt_captions = load_gt_captions(dataset["gt_captions_file"])
    
    # Evaluate
    results[dataset["name"]] = evaluate_captions(
        generated_captions, gt_captions, dataset["name"]
    )
    
# Compare results
print("\nComparison of Results:")
print(f"{'Metric':<20} {'capydata-ic':<15} {'test_uit_viic':<15}")
for metric in results["capydata-ic"]:
    print(f"{metric:<20} {results['capydata-ic'][metric]:<15.4f} {results['test_uit_viic'][metric]:<15.4f}")

Running on device: cuda


Evaluating capydata-ic: 100%|██████████| 279/279 [04:55<00:00,  1.06s/it]



Evaluation Results for capydata-ic:
Average BLEU-1: 0.5873
Average BLEU-2: 0.3728
Average BLEU-3: 0.2227
Average BLEU-4: 0.1413
Average ROUGE-L: 0.4859
Average BERTScore Precision: 0.5960
Average BERTScore Recall: 0.5779
Average BERTScore F1: 0.5853
Average CIDEr: 0.3922


Evaluating test_uit_viic: 100%|██████████| 578/578 [10:12<00:00,  1.06s/it]



Evaluation Results for test_uit_viic:
Average BLEU-1: 0.3962
Average BLEU-2: 0.1623
Average BLEU-3: 0.0715
Average BLEU-4: 0.0455
Average ROUGE-L: 0.4052
Average BERTScore Precision: 0.5075
Average BERTScore Recall: 0.5480
Average BERTScore F1: 0.5241
Average CIDEr: 0.0952

Comparison of Results:
Metric               capydata-ic     test_uit_viic  
BLEU-1               0.5873          0.3962         
BLEU-2               0.3728          0.1623         
BLEU-3               0.2227          0.0715         
BLEU-4               0.1413          0.0455         
ROUGE-L              0.4859          0.4052         
BERTScore_P          0.5960          0.5075         
BERTScore_R          0.5779          0.5480         
BERTScore_F1         0.5853          0.5241         
CIDEr                0.3922          0.0952         


## CNN + T5 

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Running on device: {device}")

# define datasets
datasets = [
    {
        "name": "capydata-ic",
        "gt_captions_file": "/kaggle/input/capydata-ic/data/test/captions.txt",
        "generated_captions_file": "/kaggle/input/test-generated-captions/generated_captions/cnn_t5/us_generated_captions.txt"
    },
    {
        "name": "test_uit_viic",
        "gt_captions_file": "/kaggle/input/uit-viic-test/test/captions.txt",
        "generated_captions_file": "/kaggle/input/test-generated-captions/generated_captions/cnn_t5/uit_generated_captions.txt"
    }
]

results = {}
for dataset in datasets:
    # Load captions
    generated_captions = load_generated_captions(dataset["generated_captions_file"])
    gt_captions = load_gt_captions(dataset["gt_captions_file"])
    
    # Evaluate
    results[dataset["name"]] = evaluate_captions(
        generated_captions, gt_captions, dataset["name"]
    )
    
# Compare results
print("\nComparison of Results:")
print(f"{'Metric':<20} {'capydata-ic':<15} {'test_uit_viic':<15}")
for metric in results["capydata-ic"]:
    print(f"{metric:<20} {results['capydata-ic'][metric]:<15.4f} {results['test_uit_viic'][metric]:<15.4f}")

Running on device: cuda


Evaluating capydata-ic: 100%|██████████| 279/279 [04:59<00:00,  1.07s/it]



Evaluation Results for capydata-ic:
Average BLEU-1: 0.5640
Average BLEU-2: 0.3263
Average BLEU-3: 0.1849
Average BLEU-4: 0.1163
Average ROUGE-L: 0.4910
Average BERTScore Precision: 0.5739
Average BERTScore Recall: 0.5577
Average BERTScore F1: 0.5643
Average CIDEr: 0.3244


Evaluating test_uit_viic: 100%|██████████| 578/578 [10:13<00:00,  1.06s/it]



Evaluation Results for test_uit_viic:
Average BLEU-1: 0.3516
Average BLEU-2: 0.1577
Average BLEU-3: 0.0707
Average BLEU-4: 0.0459
Average ROUGE-L: 0.4394
Average BERTScore Precision: 0.5141
Average BERTScore Recall: 0.5571
Average BERTScore F1: 0.5312
Average CIDEr: 0.1001

Comparison of Results:
Metric               capydata-ic     test_uit_viic  
BLEU-1               0.5640          0.3516         
BLEU-2               0.3263          0.1577         
BLEU-3               0.1849          0.0707         
BLEU-4               0.1163          0.0459         
ROUGE-L              0.4910          0.4394         
BERTScore_P          0.5739          0.5141         
BERTScore_R          0.5577          0.5571         
BERTScore_F1         0.5643          0.5312         
CIDEr                0.3244          0.1001         


## ViT + T5

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Running on device: {device}")

# define datasets
datasets = [
    {
        "name": "capydata-ic",
        "gt_captions_file": "/kaggle/input/capydata-ic/data/test/captions.txt",
        "generated_captions_file": "/kaggle/input/test-generated-captions/generated_captions/vit_t5/us_generated_captions.txt"
    },
    {
        "name": "test_uit_viic",
        "gt_captions_file": "/kaggle/input/uit-viic-test/test/captions.txt",
        "generated_captions_file": "/kaggle/input/test-generated-captions/generated_captions/vit_t5/uit_generated_captions.txt"
    }
]

results = {}
for dataset in datasets:
    # Load captions
    generated_captions = load_generated_captions(dataset["generated_captions_file"])
    gt_captions = load_gt_captions(dataset["gt_captions_file"])
    
    # Evaluate
    results[dataset["name"]] = evaluate_captions(
        generated_captions, gt_captions, dataset["name"]
    )
    
# Compare results
print("\nComparison of Results:")
print(f"{'Metric':<20} {'capydata-ic':<15} {'test_uit_viic':<15}")
for metric in results["capydata-ic"]:
    print(f"{metric:<20} {results['capydata-ic'][metric]:<15.4f} {results['test_uit_viic'][metric]:<15.4f}")

Running on device: cuda


Evaluating capydata-ic: 100%|██████████| 279/279 [04:57<00:00,  1.07s/it]



Evaluation Results for capydata-ic:
Average BLEU-1: 0.5959
Average BLEU-2: 0.3540
Average BLEU-3: 0.2053
Average BLEU-4: 0.1258
Average ROUGE-L: 0.4795
Average BERTScore Precision: 0.5866
Average BERTScore Recall: 0.5751
Average BERTScore F1: 0.5788
Average CIDEr: 0.3589


Evaluating test_uit_viic: 100%|██████████| 578/578 [10:16<00:00,  1.07s/it]



Evaluation Results for test_uit_viic:
Average BLEU-1: 0.3359
Average BLEU-2: 0.1426
Average BLEU-3: 0.0636
Average BLEU-4: 0.0404
Average ROUGE-L: 0.3923
Average BERTScore Precision: 0.4946
Average BERTScore Recall: 0.5351
Average BERTScore F1: 0.5112
Average CIDEr: 0.0973

Comparison of Results:
Metric               capydata-ic     test_uit_viic  
BLEU-1               0.5959          0.3359         
BLEU-2               0.3540          0.1426         
BLEU-3               0.2053          0.0636         
BLEU-4               0.1258          0.0404         
ROUGE-L              0.4795          0.3923         
BERTScore_P          0.5866          0.4946         
BERTScore_R          0.5751          0.5351         
BERTScore_F1         0.5788          0.5112         
CIDEr                0.3589          0.0973         


## QWEN2

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Running on device: {device}")

# define datasets
datasets = [
    {
        "name": "capydata-ic",
        "gt_captions_file": "/kaggle/input/capydata-ic/data/test/captions.txt",
        "generated_captions_file": "/kaggle/input/test-generated-captions/generated_captions/qwen2/us_generated_captions.txt"
    },
    {
        "name": "test_uit_viic",
        "gt_captions_file": "/kaggle/input/uit-viic-test/test/captions.txt",
        "generated_captions_file": "/kaggle/input/test-generated-captions/generated_captions/qwen2/uit_generated_captions.txt"
    }
]

results = {}
for dataset in datasets:
    # Load captions
    generated_captions = load_generated_captions(dataset["generated_captions_file"])
    gt_captions = load_gt_captions(dataset["gt_captions_file"])
    
    # Evaluate
    results[dataset["name"]] = evaluate_captions(
        generated_captions, gt_captions, dataset["name"]
    )
    
# Compare results
print("\nComparison of Results:")
print(f"{'Metric':<20} {'capydata-ic':<15} {'test_uit_viic':<15}")
for metric in results["capydata-ic"]:
    print(f"{metric:<20} {results['capydata-ic'][metric]:<15.4f} {results['test_uit_viic'][metric]:<15.4f}")

Running on device: cuda


Evaluating capydata-ic:  52%|█████▏    | 145/279 [02:31<02:18,  1.03s/it]

Evaluating capydata-ic: 100%|██████████| 279/279 [04:52<00:00,  1.05s/it]



Evaluation Results for capydata-ic:
Average BLEU-1: 0.6312
Average BLEU-2: 0.3763
Average BLEU-3: 0.2082
Average BLEU-4: 0.1240
Average ROUGE-L: 0.4952
Average BERTScore Precision: 0.5846
Average BERTScore Recall: 0.5764
Average BERTScore F1: 0.5788
Average CIDEr: 0.3744


Evaluating test_uit_viic: 100%|██████████| 578/578 [10:04<00:00,  1.05s/it]



Evaluation Results for test_uit_viic:
Average BLEU-1: 0.3841
Average BLEU-2: 0.1921
Average BLEU-3: 0.0919
Average BLEU-4: 0.0563
Average ROUGE-L: 0.4326
Average BERTScore Precision: 0.5124
Average BERTScore Recall: 0.5575
Average BERTScore F1: 0.5311
Average CIDEr: 0.1734

Comparison of Results:
Metric               capydata-ic     test_uit_viic  
BLEU-1               0.6312          0.3841         
BLEU-2               0.3763          0.1921         
BLEU-3               0.2082          0.0919         
BLEU-4               0.1240          0.0563         
ROUGE-L              0.4952          0.4326         
BERTScore_P          0.5846          0.5124         
BERTScore_R          0.5764          0.5575         
BERTScore_F1         0.5788          0.5311         
CIDEr                0.3744          0.1734         
